# Mutli-Run Evaluation

In [1]:
import sys
import os
# Make sure the root or 03_optimization folder is in the path
sys.path.append(os.path.abspath('../'))   # adjust path if needed
sys.path.append(os.path.abspath('../..')) 

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import re

from evaluation.evaluator import Evaluator
from evaluation.multi_run_evaluator import MultiRunEvaluator

In [4]:
#time_of_executions = ["2025-08-14_13-01", "2025-08-14_15-34"] # 3 days all models (ideal weird) => gt_freq=15min
#time_of_executions = ["2025-08-14_15-59", "2025-08-14_16-04"] # 2 days all models (gt_freq=60min)

#time_of_executions = ["2025-08-14_17-20"] # 3 days all models gt_freq=60min

#time_of_executions = ["2025-08-15_11-55", "2025-08-15_15-54", "2025-08-16_08-40", "2025-08-16_08-43", "2025-08-16_10-43", "2025-08-17_18-45"] # 7 days, mpc_f=1h, gt_f=1min, 4 models, including fixed rule-based implementation electricity_price_estimation3.csv
#time_of_executions = ["2025-08-16_12-48"] # First 15min resolution. SFH4, gt_freq=1min, 1 week, all models

#time_of_executions = ["2025-08-16_13-04", "2025-08-16_13-11"] # detection of wrong rule-base implementation
#time_of_executions = ["2025-08-16_13-16"] # with adjusted prices DImport=5, => Interval and MPCProb many bad solver results
#time_of_executions = ["2025-08-16_14-32"] # original electricity_price_estimation.csv
#time_of_executions = ["2025-08-16_14-52", "2025-08-17_08-58"] # original electricity_price_estimation.csv but 1 full week for SFH12, SFH16, SFH30 (and load result from SFH4) (mpc=1h, gt=1min)

#time_of_executions = ["2025-08-16_12-40", "2025-08-16_12-41"] # rule based comparison for different frequencies

#time_of_executions = ["2025-08-17_18-58"] # completely static prices
#time_of_executions = ["2025-08-17_19-12"] # full tou tariffs

# Checking det constraint e_end=e0
#time_of_executions = ["2025-08-18_10-39"]  # Without end constraint and electricity_price_estimation4.csv
#time_of_executions = ["2025-08-18_10-42"]  # With end constraint    and electricity_price_estimation4.csv


#time_of_executions = ["2025-08-18_09-23", "2025-08-18_09-24"] # electricity_price_estimation3.csv => rule-based strong => Also bad solver result
#time_of_executions = ["2025-08-18_09-41", "2025-08-18_09-42"] # electricity_price_estimation4.csv => ???   => Also bad solver result

# FIXED BAD SOLVER STATE WHEN e not in [e_min, e_max]
#time_of_executions = ["2025-08-18_10-14", "2025-08-18_10-15"] # redone electricity_price_estimationDImp5.csv 

# Redo estimation3 and 4 with fixed solver results
#time_of_executions = ["2025-08-18_11-17", "2025-08-18_11-18"] # electricity_price_estimation3.csv => rule-based strong 
#time_of_executions = ["2025-08-18_11-19", "2025-08-18_11-20"] # electricity_price_estimation4.csv => ???   

#time_of_executions = ["2025-08-18_12-37", "2025-08-18_12-38", "2025-08-18_12-39"]  # electricity_price_estimation5.csv

# Check other TOU tarif with greater difference
#time_of_executions = ["2025-08-18_13-59", "2025-08-18_14-00"]  # using tou_prices2.csv

# Redone electricity_price_estimation3.csv with original config.json file
#time_of_executions = ["2025-08-18_14-09", "2025-08-18_14-10"] # electricity_price_estimation3 

# Check third TOU scheme with only Export changes
#time_of_executions = ["2025-08-18_14-40", "2025-08-18_14-41"]


# Test Quadratic objective
time_of_executions = ["2025-08-19_11-43", "2025-08-19_12-42", "2025-08-19_12-45", "2025-08-19_12-46", "2025-08-19_13-34", "2025-08-19_13-43"]  # quad_prices.csv




csvs = [p for t in time_of_executions for p in Path("../runs").rglob(f"logs_{t}*.csv")]

csvs = sorted(csvs,key=lambda p: int(re.search(r"SFH(\d+)", p.name).group(1)))

mre = MultiRunEvaluator(run_paths=csvs)
mre.leaderboard()



🏢 Building: SFH4


,model,building,freq,t_start,t_end,solver_fails,pg_import_total,import_cost,pg_export_total,export_revenue,e_end,net_cost
0,ideal,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),127.534033,0.620061,-11978.368510,22.688297,2.7836,-22.068236
1,interval,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,60/10080 (0.6%),192.076996,1.003892,-12167.103889,22.220159,0.9309,-21.216267
2,mpc_det,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,60/10080 (0.6%),266.638831,1.387120,-12259.253623,22.518748,0.4873,-21.131628
3,mpc_prob,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,60/10080 (0.6%),308.190655,1.601486,-12276.397751,21.850590,0.7919,-20.249104
4,rule-based,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),0.000000,0.000000,-11896.391368,17.598643,5.7810,-17.598643



🏢 Building: SFH12


,model,building,freq,t_start,t_end,solver_fails,pg_import_total,import_cost,pg_export_total,export_revenue,e_end,net_cost
0,rule-based,SFH12,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),55.397749,0.373020,-6321.671005,11.124804,5.1132,-10.751785
1,ideal,SFH12,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),766.593643,4.687773,-6955.094609,14.828668,4.2015,-10.140895
2,interval,SFH12,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,60/10080 (0.6%),849.506414,5.736065,-7254.996387,15.172733,0.6377,-9.436668
3,mpc_prob,SFH12,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,120/10080 (1.19%),1011.858592,6.520685,-7397.215766,15.430897,1.3429,-8.910212
4,mpc_det,SFH12,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),1079.046981,6.995954,-7473.222427,15.764373,0.8187,-8.768418



🏢 Building: SFH16


,model,building,freq,t_start,t_end,solver_fails,pg_import_total,import_cost,pg_export_total,export_revenue,e_end,net_cost
0,ideal,SFH16,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),287.269168,1.490664,-10377.774518,21.400623,3.9959,-19.909959
1,interval,SFH16,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),359.918333,1.975200,-10647.385237,21.546509,0.5942,-19.571309
2,mpc_prob,SFH16,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,60/10080 (0.6%),455.016629,2.500769,-10766.258146,21.788265,0.2177,-19.287496
3,mpc_det,SFH16,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),481.683899,2.659303,-10788.863289,21.889278,0.2062,-19.229976
4,rule-based,SFH16,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),0.000000,0.000000,-10151.388473,17.609598,6.8777,-17.609598



🏢 Building: SFH30


,model,building,freq,t_start,t_end,solver_fails,pg_import_total,import_cost,pg_export_total,export_revenue,e_end,net_cost
0,ideal,SFH30,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),265.999700,1.290178,-6282.417631,14.037152,3.2093,-12.746974
1,interval,SFH30,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,60/10080 (0.6%),289.552873,1.385402,-6418.589039,13.744005,2.4262,-12.358602
2,mpc_prob,SFH30,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),424.189139,2.152603,-6623.457122,14.419255,0.8538,-12.266652
3,mpc_det,SFH30,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,60/10080 (0.6%),497.565485,2.517128,-6675.960491,14.601970,0.8617,-12.084842
4,rule-based,SFH30,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,0/10080 (0.0%),0.000000,0.000000,-5995.691843,11.788083,6.8384,-11.788083


In [4]:
mre.pivot()

model,ideal,interval,mpc_det,mpc_prob,rule-based
freq,60,60,60,60,60
building,,,,,
SFH12,-5.229742,-5.677787,-4.252453,-5.674972,-7.647397
SFH16,-12.145617,-12.344788,-10.403919,-12.141886,-12.906467
SFH30,-6.687105,-7.112538,-5.534986,-6.939654,-7.335337
SFH4,-14.112431,-14.299516,-12.657801,-14.288049,-14.465450


In [4]:
mre.failure_table()

,model,building,freq,t_start,t_end,steps,solver_fail_count,solver_fail_pct,solver_status_mix,solver_top_errors,path
0,ideal,SFH16,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,120,0.011905,error1:120,fallback_decision:120,../runs/2025-08-16_10-43/SFH16/logs_2025-08-16...
1,ideal,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_11-55/SFH4/logs_2025-08-15_...
2,interval,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_11-55/SFH4/logs_2025-08-15_...
3,mpc_det,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_11-55/SFH4/logs_2025-08-15_...
4,mpc_prob,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_11-55/SFH4/logs_2025-08-15_...
5,rule-based,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_15-54/SFH4/logs_2025-08-15_...
6,ideal,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_15-54/SFH4/logs_2025-08-15_...
7,ideal,SFH4,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-16_08-43/SFH4/logs_2025-08-16_...
8,mpc_prob,SFH12,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_11-55/SFH12/logs_2025-08-15...
9,interval,SFH12,60,2020-06-07 04:00:00+00:00,2020-06-14 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-15_11-55/SFH12/logs_2025-08-15...


## Test the evaluation class

In [ ]:
run_path = Path('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv')
run_path_rb = Path('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')

prices_path = Path('../../01_data/price_data/electricity_price_estimation.csv')
prices_path_const = Path('../../01_data/price_data/const_prices_testing.csv')
ev1 = Evaluator(run_path, prices_path)
ev1.get_costs()

{'import_cost': 0.0,
 'export_revenue': 2.344861855120558,
 'net_cost': -2.344861855120558}

In [7]:
ev2 = Evaluator(run_path_rb, prices_path)
ev2.get_costs()

{'import_cost': 0.0,
 'export_revenue': 2.186811788658414,
 'net_cost': -2.186811788658414}

In [8]:
ev1.get_df()

,action,pb_low,pb_high,pg,gt,soe_now,soe_new,import_price,export_price,costs_buy,costs_sell,cashflow
timestamp,,,,,,,,,,,,
2020-05-13 07:00:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.511872,-2.834009,4.000000,4.076507,0.1139,0.0465,0.0,0.116802,-0.116802
2020-05-13 07:15:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.557424,-2.879560,4.076507,4.153015,0.1139,0.0465,0.0,0.118920,-0.118920
2020-05-13 07:30:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.854330,-3.176466,4.153015,4.229522,0.1139,0.0465,0.0,0.132726,-0.132726
2020-05-13 07:45:00+00:00,-0.322137,-0.322137,-1.201852e-07,-3.387559,-3.709695,4.229522,4.306030,0.1139,0.0465,0.0,0.157521,-0.157521
2020-05-13 08:00:00+00:00,0.655249,0.655249,1.088511e+00,-4.045914,-3.390665,4.306030,4.133596,0.1138,0.0465,0.0,0.188135,-0.188135
2020-05-13 08:15:00+00:00,0.655249,0.655249,1.088511e+00,-4.228103,-3.572853,4.133596,3.961162,0.1138,0.0465,0.0,0.196607,-0.196607
2020-05-13 08:30:00+00:00,0.655249,0.655249,1.088511e+00,-5.395369,-4.740119,3.961162,3.788728,0.1138,0.0465,0.0,0.250885,-0.250885
2020-05-13 08:45:00+00:00,0.655249,0.655249,1.088511e+00,-5.698039,-5.042790,3.788728,3.616294,0.1138,0.0465,0.0,0.264959,-0.264959
2020-05-13 09:00:00+00:00,-0.556352,-0.556352,-1.791756e-06,-4.561881,-5.118234,3.616294,3.748427,0.1094,0.0448,0.0,0.204372,-0.204372


In [9]:
ev1.plot_battery_soe(max_soe=8.0)

In [10]:
ev2.plot_battery_soe(max_soe=8.0)

## Test Multi-Run Evaluator

In [11]:
path1 = Path('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv')
path2 = Path('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')

path_list = [path1, path2]

mre = MultiRunEvaluator(path_list)

self.run_paths: [WindowsPath('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv'), WindowsPath('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')]


In [12]:
mre.leaderboard(by="net_cost")

,model,building,freq,t_start,t_end,net_cost
0,interval,SFH3,60,2020-05-13 07:00:00+00:00,2020-05-13 09:45:00+00:00,-2.344862
1,rule-based,SFH3,60,2020-05-13 07:00:00+00:00,2020-05-13 09:45:00+00:00,-2.186812


In [13]:
mre.pivot()

model,interval,rule-based
freq,60,60
building,,
SFH3,-2.344862,-2.186812
